# FloPy3

### Quick demo on how FloPy3 handles external files for arrays

In [1]:
import os 
import shutil
import flopy
import numpy as np

In [2]:
# make a model
nlay,nrow,ncol = 10,20,5
model_ws = os.path.join("data","external_demo")

if os.path.exists(model_ws):
    shutil.rmtree(model_ws)

# the place for all of your hand made and costly model inputs    
array_dir = os.path.join("data","array_dir")
if os.path.exists(array_dir):
    shutil.rmtree(array_dir)    
os.mkdir(array_dir)    
    
ml = flopy.modflow.Modflow(model_ws=model_ws)
dis = flopy.modflow.ModflowDis(ml,nlay=nlay,nrow=nrow,ncol=ncol,steady=False,nper=2)

make an ``hk`` and ```vka``` array.  We'll save ```hk``` to files - pretent that you spent months making this important model property.  Then make an ```lpf```

In [3]:
hk = np.zeros((nlay,nrow,ncol)) + 5.0
vka = np.zeros_like(hk)
fnames = []
for i,h in enumerate(hk):
    fname = os.path.join(array_dir,"hk_{0}.ref".format(i+1))
    fnames.append(fname)
    np.savetxt(fname,h)
    vka[i] = i+1
lpf = flopy.modflow.ModflowLpf(ml,hk=fnames,vka=vka)   

Let's also have some recharge with mixed args as well.  Pretent the recharge in the second stress period is very important and precise

In [4]:
warmup_recharge = np.ones((nrow,ncol))
important_recharge = np.random.random((nrow,ncol))
fname = os.path.join(array_dir,"important_recharge.ref")
np.savetxt(fname,important_recharge)
rch = flopy.modflow.ModflowRch(ml,rech={0:warmup_recharge,1:fname})


In [5]:
ml.write_input()

Let's look at the files that were created

In [6]:
print("model_ws:",ml.model_ws)
print('\n'.join(os.listdir(ml.model_ws)))

('model_ws:', 'data/external_demo')
hk_1.ref
hk_10.ref
hk_2.ref
hk_3.ref
hk_4.ref
hk_5.ref
hk_6.ref
hk_7.ref
hk_8.ref
hk_9.ref
important_recharge.ref
modflowtest.dis
modflowtest.lpf
modflowtest.nam
modflowtest.rch


We see that a copy of the ``hk`` files as well as the important recharge file were made in the ```model_ws```.Let's looks at the ```lpf``` file

In [7]:
open(os.path.join(ml.model_ws,ml.name+".lpf"),'r').readlines()[:20]

['# LPF for MODFLOW, generated by Flopy.\n',
 '        53    -1E+30         0  \n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '              1              1              1              1              1              1              1              1              1              1\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 'OPEN/CLOSE                        hk_1.ref               1   (5G15.6) -1 hk layer 1                    \n',
 'INTERNAL               1   (5G15.6) -1 #vka layer 1                   \n',
 '              1              1              1              1              1\n',
 '              1              1              1              1

We see that the ```open/close``` approach was used - this is because ``ml.free_format`` is ``True``.  Notice that ```vka``` is written internally

In [8]:
ml.free_format

True

Now change ```model_ws```

In [9]:
print(ml.model_ws)
ml.model_ws = os.path.join("data","new_external_demo_dir")

data/external_demo

changing model workspace...
   data/new_external_demo_dir


Now when we call ``write_input()``, a copy of external files are made in the current ```model_ws```

In [10]:
ml.write_input()

In [11]:
# list the files in model_ws that have 'hk' in the name
print('\n'.join([name for name in os.listdir(ml.model_ws) if "hk" in name or "impor" in name]))

hk_1.ref
hk_10.ref
hk_2.ref
hk_3.ref
hk_4.ref
hk_5.ref
hk_6.ref
hk_7.ref
hk_8.ref
hk_9.ref
important_recharge.ref


Now we see that the external files were copied to the new ```model_ws```

###Using ```external_path```

It is sometimes useful when first building a model to write the model arrays as external files for processing and parameter estimation.  The ```model``` attribute ```external_path``` triggers this behavior

In [12]:
# make a model - same code as before except for the model constructor
nlay,nrow,ncol = 10,20,5
model_ws = os.path.join("data","external_demo")

if os.path.exists(model_ws):
    shutil.rmtree(model_ws)

# the place for all of your hand made and costly model inputs    
array_dir = os.path.join("data","array_dir")
if os.path.exists(array_dir):
    shutil.rmtree(array_dir)    
os.mkdir(array_dir)    

# lets make an external path relative to the model_ws
ml = flopy.modflow.Modflow(model_ws=model_ws, external_path="ref")
dis = flopy.modflow.ModflowDis(ml,nlay=nlay,nrow=nrow,ncol=ncol,steady=False,nper=2)

hk = np.zeros((nlay,nrow,ncol)) + 5.0
vka = np.zeros_like(hk)
fnames = []
for i,h in enumerate(hk):
    fname = os.path.join(array_dir,"hk_{0}.ref".format(i+1))
    fnames.append(fname)
    np.savetxt(fname,h)
    vka[i] = i+1
lpf = flopy.modflow.ModflowLpf(ml,hk=fnames,vka=vka) 

warmup_recharge = np.ones((nrow,ncol))
important_recharge = np.random.random((nrow,ncol))
fname = os.path.join(array_dir,"important_recharge.ref")
np.savetxt(fname,important_recharge)
rch = flopy.modflow.ModflowRch(ml,rech={0:warmup_recharge,1:fname})

We can see that the model constructor created both ```model_ws``` and ```external_path``` which is _relative to the model_ws_

In [13]:
os.listdir(ml.model_ws)

['ref']

Now, when we call ```write_input()```, any array properties that were specified as ```np.ndarray``` will be written externally.  If a scalar was passed as the argument, the value remains internal to the model input files

In [14]:
ml.write_input()
open(os.path.join(ml.model_ws,ml.name+".lpf"),'r').readlines()[:20]

['# LPF for MODFLOW, generated by Flopy.\n',
 '        53    -1E+30         0  \n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '              1              1              1              1              1              1              1              1              1              1\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 'OPEN/CLOSE                    ref/hk_1.ref               1   (5G15.6) -1 hk layer 1                    \n',
 'OPEN/CLOSE             ref/vka_layer_1.ref               1   (5G15.6) -1 vka layer 1                   \n',
 'CONSTANT           1E-05                           #ss layer 1                    \n',
 'OPEN/CLOSE          

Now, ```vka``` was also written externally, but not the storage properties.Let's verify the contents of the external path directory. We see our hard-fought ```hk``` and ```important_recharge``` arrays, as well as the ``vka`` arrays.

In [15]:
ml.lpf.ss.how = "internal"
ml.write_input()
open(os.path.join(ml.model_ws,ml.name+".lpf"),'r').readlines()[:20]

['# LPF for MODFLOW, generated by Flopy.\n',
 '        53    -1E+30         0  \n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '              1              1              1              1              1              1              1              1              1              1\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 'OPEN/CLOSE                    ref/hk_1.ref               1   (5G15.6) -1 hk layer 1                    \n',
 'OPEN/CLOSE             ref/vka_layer_1.ref               1   (5G15.6) -1 vka layer 1                   \n',
 'INTERNAL               1   (5G15.6) -1 #ss layer 1                    \n',
 '          1E-05          1E-05  

In [16]:
print('\n'.join(os.listdir(os.path.join(ml.model_ws,ml.external_path))))

hk_1.ref
hk_10.ref
hk_2.ref
hk_3.ref
hk_4.ref
hk_5.ref
hk_6.ref
hk_7.ref
hk_8.ref
hk_9.ref
important_recharge.ref
rech_0.ref
vka_layer_1.ref
vka_layer_10.ref
vka_layer_2.ref
vka_layer_3.ref
vka_layer_4.ref
vka_layer_5.ref
vka_layer_6.ref
vka_layer_7.ref
vka_layer_8.ref
vka_layer_9.ref


###Fixed format

All of this behavior also works for fixed-format type models (really, really old models - I mean OLD!)

In [17]:
# make a model - same code as before except for the model constructor
nlay,nrow,ncol = 10,20,5
model_ws = os.path.join("data","external_demo")

if os.path.exists(model_ws):
    shutil.rmtree(model_ws)

# the place for all of your hand made and costly model inputs    
array_dir = os.path.join("data","array_dir")
if os.path.exists(array_dir):
    shutil.rmtree(array_dir)    
os.mkdir(array_dir)    

# lets make an external path relative to the model_ws
ml = flopy.modflow.Modflow(model_ws=model_ws, external_path="ref")

# explicitly reset the free_format flag
ml.free_format = False

dis = flopy.modflow.ModflowDis(ml,nlay=nlay,nrow=nrow,ncol=ncol,steady=False,nper=2)

hk = np.zeros((nlay,nrow,ncol)) + 5.0
vka = np.zeros_like(hk)
fnames = []
for i,h in enumerate(hk):
    fname = os.path.join(array_dir,"hk_{0}.ref".format(i+1))
    fnames.append(fname)
    np.savetxt(fname,h)
    vka[i] = i+1
lpf = flopy.modflow.ModflowLpf(ml,hk=fnames,vka=vka)
ml.lpf.ss.how = "internal"
warmup_recharge = np.ones((nrow,ncol))
important_recharge = np.random.random((nrow,ncol))
fname = os.path.join(array_dir,"important_recharge.ref")
np.savetxt(fname,important_recharge)
rch = flopy.modflow.ModflowRch(ml,rech={0:warmup_recharge,1:fname})

ml.write_input()

We see that now the external arrays are being handled through the name file.  Let's look at the name file

In [18]:
open(os.path.join(ml.model_ws,ml.name+".nam"),'r').readlines()

['# Name file for mf2005, generated by Flopy.\n',
 'LIST           2 modflowtest.list\n',
 'DIS           11 modflowtest.dis \n',
 'LPF           15 modflowtest.lpf \n',
 'RCH           19 modflowtest.rch \n',
 'DATA          1001  ref/hk_1.ref\n',
 'DATA          1002  ref/vka_layer_1.ref\n',
 'DATA          1003  ref/hk_2.ref\n',
 'DATA          1004  ref/vka_layer_2.ref\n',
 'DATA          1005  ref/hk_3.ref\n',
 'DATA          1006  ref/vka_layer_3.ref\n',
 'DATA          1007  ref/hk_4.ref\n',
 'DATA          1008  ref/vka_layer_4.ref\n',
 'DATA          1009  ref/hk_5.ref\n',
 'DATA          1010  ref/vka_layer_5.ref\n',
 'DATA          1011  ref/hk_6.ref\n',
 'DATA          1012  ref/vka_layer_6.ref\n',
 'DATA          1013  ref/hk_7.ref\n',
 'DATA          1014  ref/vka_layer_7.ref\n',
 'DATA          1015  ref/hk_8.ref\n',
 'DATA          1016  ref/vka_layer_8.ref\n',
 'DATA          1017  ref/hk_9.ref\n',
 'DATA          1018  ref/vka_layer_9.ref\n',
 'DATA          1019  ref

###"free" and "binary" format

In [18]:
ml.dis.botm[0].format.binary = True
ml.write_input()

Util2d:botm layer 1 warning: resetting how to external since format is binary
BaseModel.add_external() warning: replacing existing filename ref/hk_1.ref
BaseModel.add_external() warning: replacing existing filename ref/vka_layer_1.ref
BaseModel.add_external() warning: replacing existing filename ref/hk_2.ref
BaseModel.add_external() warning: replacing existing filename ref/vka_layer_2.ref
BaseModel.add_external() warning: replacing existing filename ref/hk_3.ref
BaseModel.add_external() warning: replacing existing filename ref/vka_layer_3.ref
BaseModel.add_external() warning: replacing existing filename ref/hk_4.ref
BaseModel.add_external() warning: replacing existing filename ref/vka_layer_4.ref
BaseModel.add_external() warning: replacing existing filename ref/hk_5.ref
BaseModel.add_external() warning: replacing existing filename ref/vka_layer_5.ref
BaseModel.add_external() warning: replacing existing filename ref/hk_6.ref
BaseModel.add_external() warning: replacing existing filename 

In [19]:
open(os.path.join(ml.model_ws,ml.name+".nam"),'r').readlines()

['# Name file for mf2005, generated by Flopy.\n',
 'LIST           2 modflowtest.list\n',
 'DIS           11 modflowtest.dis \n',
 'LPF           15 modflowtest.lpf \n',
 'RCH           19 modflowtest.rch \n',
 'DATA(BINARY)  -1023  ref/botm_layer_1.ref REPLACE\n',
 'DATA          1024  ref/hk_1.ref\n',
 'DATA          1025  ref/vka_layer_1.ref\n',
 'DATA          1026  ref/hk_2.ref\n',
 'DATA          1027  ref/vka_layer_2.ref\n',
 'DATA          1028  ref/hk_3.ref\n',
 'DATA          1029  ref/vka_layer_3.ref\n',
 'DATA          1030  ref/hk_4.ref\n',
 'DATA          1031  ref/vka_layer_4.ref\n',
 'DATA          1032  ref/hk_5.ref\n',
 'DATA          1033  ref/vka_layer_5.ref\n',
 'DATA          1034  ref/hk_6.ref\n',
 'DATA          1035  ref/vka_layer_6.ref\n',
 'DATA          1036  ref/hk_7.ref\n',
 'DATA          1037  ref/vka_layer_7.ref\n',
 'DATA          1038  ref/hk_8.ref\n',
 'DATA          1039  ref/vka_layer_8.ref\n',
 'DATA          1040  ref/hk_9.ref\n',
 'DATA         

In [20]:
open(os.path.join(ml.model_ws,ml.name+".dis"),'r').readlines()

['# Discretization file for MODFLOW, generated by Flopy.\n',
 '        10        20         5         2         4         2\n',
 '  0  0  0  0  0  0  0  0  0  0\n',
 '         0         1            (5G15.6)        -1 #delr\n',
 '         0         1           (20G15.6)        -1 #delc\n',
 '         0         1            (5G15.6)        -1 #model_top\n',
 '     -1023         1            (BINARY)        -1 #botm layer 1\n',
 '         0         1            (5G15.6)        -1 #botm layer 2\n',
 '         0         1            (5G15.6)        -1 #botm layer 3\n',
 '         0         1            (5G15.6)        -1 #botm layer 4\n',
 '         0         1            (5G15.6)        -1 #botm layer 5\n',
 '         0         1            (5G15.6)        -1 #botm layer 6\n',
 '         0         1            (5G15.6)        -1 #botm layer 7\n',
 '         0         1            (5G15.6)        -1 #botm layer 8\n',
 '         0         1            (5G15.6)        -1 #botm layer 9\n',
 '

###The ```.how``` attribute
```Util2d``` includes a ```.how``` attribute that gives finer grained control of how arrays will written

In [22]:
ml.free_format = True
ml.lpf.hk[0].how

'external'

In [23]:
ml.lpf.hk[0].how = "openclose"
ml.lpf.hk[0].how

'openclose'

In [24]:
ml.dis.top.how

'constant'

In [25]:
ml.dis.top.how = "external"

In [26]:
ml.write_input()

Util2d:botm layer 1 warning: resetting how to external since format is binary
BaseModel.add_external() warning: replacing existing filename ref/vka_layer_1.ref
BaseModel.add_external() warning: replacing existing filename ref/hk_2.ref
BaseModel.add_external() warning: replacing existing filename ref/vka_layer_2.ref
BaseModel.add_external() warning: replacing existing filename ref/hk_3.ref
BaseModel.add_external() warning: replacing existing filename ref/vka_layer_3.ref
BaseModel.add_external() warning: replacing existing filename ref/hk_4.ref
BaseModel.add_external() warning: replacing existing filename ref/vka_layer_4.ref
BaseModel.add_external() warning: replacing existing filename ref/hk_5.ref
BaseModel.add_external() warning: replacing existing filename ref/vka_layer_5.ref
BaseModel.add_external() warning: replacing existing filename ref/hk_6.ref
BaseModel.add_external() warning: replacing existing filename ref/vka_layer_6.ref
BaseModel.add_external() warning: replacing existing fi

In [27]:
open(os.path.join(ml.model_ws,ml.name+".dis"),'r').readlines()

['# Discretization file for MODFLOW, generated by Flopy.\n',
 '        10        20         5         2         4         2\n',
 '  0  0  0  0  0  0  0  0  0  0\n',
 'CONSTANT               1                           #delr                          \n',
 'CONSTANT               1                           #delc                          \n',
 'EXTERNAL                            1046               1   (5G15.6) -1 model_top                     \n',
 'OPEN/CLOSE            ref/botm_layer_1.ref               1   (BINARY) -1 botm layer 1                  \n',
 'CONSTANT               0                           #botm layer 2                  \n',
 'CONSTANT               0                           #botm layer 3                  \n',
 'CONSTANT               0                           #botm layer 4                  \n',
 'CONSTANT               0                           #botm layer 5                  \n',
 'CONSTANT               0                           #botm layer 6                 

In [28]:
open(os.path.join(ml.model_ws,ml.name+".lpf"),'r').readlines()

['# LPF for MODFLOW, generated by Flopy.\n',
 '        53    -1E+30         0  \n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '              1              1              1              1              1              1              1              1              1              1\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 'OPEN/CLOSE                    ref/hk_1.ref               1   (5G15.6) -1 hk layer 1                    \n',
 'EXTERNAL                            1047               1   (5G15.6) -1 vka layer 1                   \n',
 'INTERNAL               1   (5G15.6) -1 #ss layer 1                    \n',
 '          1E-05          1E-05    

In [29]:
open(os.path.join(ml.model_ws,ml.name+".nam"),'r').readlines()

['# Name file for mf2005, generated by Flopy.\n',
 'LIST           2 modflowtest.list\n',
 'DIS           11 modflowtest.dis \n',
 'LPF           15 modflowtest.lpf \n',
 'RCH           19 modflowtest.rch \n',
 'DATA(BINARY)  -1023  ref/botm_layer_1.ref REPLACE\n',
 'DATA          1024  ref/hk_1.ref\n',
 'DATA          1046  ref/model_top.ref\n',
 'DATA          1047  ref/vka_layer_1.ref\n',
 'DATA          1048  ref/hk_2.ref\n',
 'DATA          1049  ref/vka_layer_2.ref\n',
 'DATA          1050  ref/hk_3.ref\n',
 'DATA          1051  ref/vka_layer_3.ref\n',
 'DATA          1052  ref/hk_4.ref\n',
 'DATA          1053  ref/vka_layer_4.ref\n',
 'DATA          1054  ref/hk_5.ref\n',
 'DATA          1055  ref/vka_layer_5.ref\n',
 'DATA          1056  ref/hk_6.ref\n',
 'DATA          1057  ref/vka_layer_6.ref\n',
 'DATA          1058  ref/hk_7.ref\n',
 'DATA          1059  ref/vka_layer_7.ref\n',
 'DATA          1060  ref/hk_8.ref\n',
 'DATA          1061  ref/vka_layer_8.ref\n',
 'DATA    